## Step1: 导入必要的包

In [1]:
import os
from multiprocessing import cpu_count
import numpy as np
import shutil
import paddle
import paddle.fluid as fluid

- paddle.fluid 飞桨核心框架
- multiprocessing python中多进程管理包
- numpy 科学计算模块
- shutil 文本操作模块
- os 操作系统模块

## Step2: 准备数据

**数据集**：
- 爬虫爬取中文文本数据
- 爬取56821条数据中文新闻标题
- 10个类别：国际、文化、娱乐、体育、财经、汽车、教育、科技、房产、证券

In [2]:
# 设置各类数据文件目录
data_root_path = './text_data/'
dict_path = os.path.join(data_root_path, 'dict_txt.txt')
data_path = os.path.join(data_root_path, 'news_classify_data.txt')

### 2.1 创建数据字典

In [3]:
def create_dict(data_path, dict_path):
    # 初始化容器集合
    dict_set = set()
    
    # 按行读取原始数据
    with open(data_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    for line in lines:
        # 获取当前新闻的标题
        title = line.split('_!_')[-1].replace('\n', "")
        
        # 将当前新闻标题中的每个字依次存放到集合容器中
        for s in title:
            dict_set.add(s)
    
    # 初始化字典列表
    dict_list = []
    
    # 初始化序号
    i = 0
    
    # 给集合容器中的每个字分配序号依次存入到字典列表中
    for s in dict_set:
        dict_list.append([s, i])
        i += 1
        
    # 根据字典列表生成字典    
    dict_txt = dict(dict_list)
    
    # 最后手动加入字典 {<unk> : end_id}
    end_dict = {"<unk>": i}
    dict_txt.update(end_dict)
    
    # 将生成好的数据字典写入文件保存
    with open(dict_path, 'w', encoding='utf-8') as f:
        f.write(str(dict_txt))
    
    print("数据字典生成完成！")

In [4]:
create_dict(data_path, dict_path)

数据字典生成完成！


### 2.2 创建数据集

In [9]:
# 根据原始数据集创建序列化表示的数据
def create_data_list(data_root_path):
    # 新建测试文件数据集
    with open(data_root_path + 'test_list.txt', 'w') as f:
        pass
    
    # 新建训练文件数据集
    with open(data_root_path + 'train_list.txt', 'w') as f:
        pass
    
    # 按行读取数据字典文件并转换为字典类型
    with open(dict_path, 'r', encoding='utf-8') as f_data:
        dict_txt = eval(f_data.readlines()[0])
    
    # 按行读取原始数据
    with open(data_path, 'r', encoding = 'utf-8') as f_data:
        lines = f_data.readlines()
    
    # 初始化序号
    i = 0
    
    # 按行读取原始数据
    for line in lines:
        # 获取当前数据新闻标题
        title = line.split('_!_')[-1].replace('\n', '')
        
        # 获取当前数据新闻分类标签
        lab = line.split('_!_')[1]
        
        # 初始化当前数据新闻标题数字ids
        title_ids = ""
        
        # 每10条数据选取一条作为测试用数据
        if i % 10 == 0:
            with open(os.path.join(data_root_path, 'test_list.txt'), 'a', encoding='utf-8') as f_test:
                for word in title:
                    # 获取当前标题中每个字在数据字典中对应的id
                    word_id = str(dict_txt[word])
                    
                    # 拼接每一个字对应的id形成标题的完整ids，中间用逗号隔开
                    title_ids = title_ids + word_id + ','
                
                # 拼接完成当前新闻标题的数字ids后，去掉末尾的逗号
                title_ids = title_ids[:-1]
                
                # 将当前新闻标题的数字ids 和 新闻分类标签拼接起来，中间用 table 隔开
                title_ids = title_ids + '\t' + lab + '\n'
                
                # 写入测试文件列表
                f_test.write(title_ids)
                
        else:
            # 其他数据作为训练数据
            with open(os.path.join(data_root_path, 'train_list.txt'), 'a', encoding='utf-8') as f_train:
                for word in title:
                    # 获取当前标题中每个字在数据字典中对应的id
                    word_id = str(dict_txt[word])
                    
                    # 拼接每一个字对应的id形成标题的完整ids，中间用逗号隔开
                    title_ids = title_ids + word_id + ','
                
                # 拼接完成当前新闻标题的数字ids后，去掉末尾的逗号
                title_ids = title_ids[:-1]
                
                # 将当前新闻标题的数字ids 和 新闻分类标签拼接起来，中间用 table 隔开
                title_ids = title_ids + '\t' + lab + '\n'
                
                # 写入训练文件列表
                f_train.write(title_ids)
        
        i += 1
    
    print('数据列表生成完成！')  

In [10]:
create_data_list(data_root_path)

数据列表生成完成！


### 2.3 训练/测试数据的预处理

创建data_mapper

In [4]:
def data_mapper(sample):
    data, label = sample
    data = [int(d) for d in data.split(',')]
    
    return data, int(label)

创建训练数据提取器 train_reader

In [5]:
def train_reader(train_list_path):
    
    def reader():
        # 按行从训练数据列表中读取数据
        with open(train_list_path, 'r') as f:
            lines = f.readlines()
            
            # 打乱数据
            np.random.shuffle(lines)
            
            # 获取每条新闻标题的数字ids和分类标签
            for line in lines:
                data, label = line.split('\t')
                yield data, label
                
    return paddle.reader.xmap_readers(data_mapper, reader, cpu_count(), 1024)

创建测试数据提取器 test_reader

In [6]:
def test_reader(test_list_path):
    
    def reader():
        with open(test_list_path, 'r') as f:
            lines = f.readlines()
            
            for line in lines:
                data, label = line.split('\t')
                yield data, label
                
    return paddle.reader.xmap_readers(data_mapper, reader, cpu_count(), 1024)

paddle.reader.xmap_readers():通过多线程方式，通过用户自定义的映射器mapper来映射reader返回的样本（到输出队列)。

## Step3: 配置神经网络 

**卷积神经网络（Convolutional Neural Networks, CNN）**

输入词向量序列，产生一个特征图（feature map），对特征图采用时间维度上的最大池化（max pooling over time）操作得到此卷积核对应的整句话的特征，最后，将所有卷积核得到的特征拼接起来即为文本的定长向量表示，对于文本分类问题，将其连接至softmax即构建出完整的模型。

在实际应用中，使用多个卷积核来处理句子，窗口大小相同的卷积核堆叠起来形成一个矩阵，这样可以更高效的完成运算。

另外，也可使用窗口大小不同的卷积核来处理句子。

![](https://github.com/pchen12567/picture_store/blob/master/PaddlePaddle/text_classify_001.png?raw=true)

### 3.1 创建CNN网络

定义的网络结构是：输入层-->卷积与池化层-->输出层

In [7]:
def CNN_net(data, dict_dim, class_dim=10, emb_dim=128, hid_dim=128, hid_dim2=98):
    emb = fluid.layers.embedding(input=data, size= [dict_dim, emb_dim])
    
    conv_3 = fluid.nets.sequence_conv_pool(
                                            input=emb,
                                            num_filters=hid_dim,
                                            filter_size=3,
                                            act='tanh',
                                            pool_type='sqrt')
    
    
    conv_4 = fluid.nets.sequence_conv_pool(
                                            input=emb,
                                            num_filters=hid_dim2,
                                            filter_size=4,
                                            act='tanh',
                                            pool_type='sqrt')
    
    output = fluid.layers.fc(
                            input=[conv_3, conv_4],
                            size=class_dim,
                            act='softmax')
    
    return output

### 3.2 定义输入数据

- 张量words: 数据类型为int64, lod_level不为0则输入数据为序列数据
- 张量label: 代表文本分类后的类别，形状为[1]，数据类型为int64

In [8]:
words = fluid.layers.data(name='words', shape=[1], dtype='int64', lod_level=1)
label = fluid.layers.data(name='label', shape=[1], dtype='int64')

### 3.3 获取分类器

In [9]:
# 获取字典的长度
def get_dict_len(dict_path):
    with open(dict_path, 'r', encoding='utf-8') as f:
        # eval()函数将str转换为dict
        line = eval(f.readlines()[0])
    
    return len(line.keys())

In [10]:
dict_dim = get_dict_len(dict_path)

In [11]:
# 获取分类器
model = CNN_net(words, dict_dim)

### 3.4 定义损失函数

- 交叉熵损失函数在分类任务上比较常用
- 定义了一个损失函数之后，还要求平均值，因为定义的是一个batch的损失值
- 同时定义一个准确率函数，可以在训练的试函输出分类的准确率

In [12]:
# 获取损失函数和准确率函数
cost = fluid.layers.cross_entropy(input=model, label=label)
avg_cost = fluid.layers.mean(cost)
acc = fluid.layers.accuracy(input=model, label=label)

### 3.5 定义优化算法

**Adagrad优化器**
- 能够在训练中自动的对学习率进行调整，对于出现频率较低参数采用较大的学习了更新；
- 相反，对于出现频率较高的参数采用较小的学习了更新，适合出来稀疏数据。

In [13]:
optimizer = fluid.optimizer.AdagradOptimizer(learning_rate=0.002)
opt = optimizer.minimize(avg_cost)

上述模型配置完毕后，得到两个fluid.program

- fluid.default_startup_program():
    - 参数初始化操作会被写入到 fluid.default_startup_program() 中
- fluid.default_main_program():
    - 用于获取默认或全局 main program
    - 该主程序用于训练和测试模型，fluid.layers 中的所有layer函数可以向 default_main_program 中添加算子和变量。
    - 是 fluid 许多编程接口的缺省值

### 3.6 获取预测程序

In [14]:
test_program = fluid.default_main_program().clone(for_test=True)

## Step4: 训练模型

### 4.1 创建训练用执行器

- 指定程序运行的设备，fluid.CPUPlace()和fluid.CUDAPlace()分别表示为CPU和GPU
- 创建一个Executor实例
- Executor 接收传入的 program，并通过run()方法运行program

In [15]:
# 创建一个执行器，CPU训练速度比较慢
place = fluid.CPUPlace()  # CPU
# place = fluid.CUDAPlace(0)  # GPU
exe = fluid.Executor(place)

# 进行参数初始化
exe.run(fluid.default_startup_program())

[]

### 4.2 定义数据映射器

In [16]:
feeder = fluid.DataFeeder(place=place, feed_list=[words, label])

### 4.3 获取训练和测试数据读取器

In [17]:
train_list_path = './text_data/train_list.txt'
test_list_path = './text_data/test_list.txt'

In [18]:
train_reader = paddle.batch(reader=train_reader(train_list_path), batch_size=128)
test_reader = paddle.batch(reader=test_reader(test_list_path), batch_size=128)

### 4.4 开始训练并测试

In [19]:
# 设置训练轮数
EPOCH_NUM = 10

# 设置模型保存路径
model_save_dir = './clssify_model/'

In [20]:
for epoch_id in range(EPOCH_NUM):
    
    # 进行训练
    for batch_id, data in enumerate(train_reader()):
        # 对于 train_reader 中的每次batch，执行exe.run()运行执行器训练
        # 喂入每个batch的训练数据，fetch损失值、准确率
        train_cost, train_acc = exe.run(
                                        program = fluid.default_main_program(),
                                        feed = feeder.feed(data),
                                        fetch_list = [avg_cost, acc])
        
        # 每100个batch打印一次训练结果，一个batch包含128条数据
        if batch_id % 50 == 0:
            print('Epoch: {}, Batch: {}, Cost: {:.4f}, Acc: {:.4f}'.format(epoch_id, batch_id, float(train_cost), float(train_acc)))
    
    # 进行测试
    test_costs = []
    test_accs = []
    
    for batch_id, data in enumerate(test_reader()):
        # 对于 test_reader 中的每次batch，执行exe.run()运行执行器训练
        # 喂入每个batch的训练数据，fetch损失值、准确率
        test_cost, test_acc = exe.run(
                                        program = test_program,
                                        feed = feeder.feed(data),
                                        fetch_list = [avg_cost, acc])
        
        test_costs.append(test_cost[0])
        test_accs.append(test_acc[0])
        
    # 计算每轮所有batch的误差平均值、误差准确率，然后输出   
    test_cost = (sum(test_costs) / len(test_costs))
    test_acc = (sum(test_accs) / len(test_accs))
    print('Test: {}, Cost: {:.4f}, Acc: {:.4f}'.format(epoch_id, float(test_cost), float(test_acc)))
    print("*************************************")

Epoch: 0, Batch: 0, Cost: 2.2994, Acc: 0.1094
Epoch: 0, Batch: 50, Cost: 1.3757, Acc: 0.6328
Epoch: 0, Batch: 100, Cost: 1.0778, Acc: 0.6953
Epoch: 0, Batch: 150, Cost: 0.9665, Acc: 0.7188
Epoch: 0, Batch: 200, Cost: 0.9725, Acc: 0.7109
Epoch: 0, Batch: 250, Cost: 0.7764, Acc: 0.7734
Epoch: 0, Batch: 300, Cost: 0.9397, Acc: 0.6875
Epoch: 0, Batch: 350, Cost: 0.7377, Acc: 0.7188
Test: 0, Cost: 0.8139, Acc: 0.7391
*************************************
Epoch: 1, Batch: 0, Cost: 0.7662, Acc: 0.7344
Epoch: 1, Batch: 50, Cost: 0.7176, Acc: 0.7656
Epoch: 1, Batch: 100, Cost: 0.7560, Acc: 0.7344
Epoch: 1, Batch: 150, Cost: 0.7311, Acc: 0.7266
Epoch: 1, Batch: 200, Cost: 0.6404, Acc: 0.7891
Epoch: 1, Batch: 250, Cost: 0.6908, Acc: 0.7656
Epoch: 1, Batch: 300, Cost: 0.9653, Acc: 0.7109
Epoch: 1, Batch: 350, Cost: 0.8799, Acc: 0.7344
Test: 1, Cost: 0.7545, Acc: 0.7541
*************************************
Epoch: 2, Batch: 0, Cost: 0.5530, Acc: 0.8125
Epoch: 2, Batch: 50, Cost: 0.6356, Acc: 0.8125

### 4.5 保存模型

参数说明：
- dirname(str): 保存预测模型的路径
- feeded_var_names(list[str])：预测需要feed的数据
- target_vars(list[variable])：保存预测结果的变量
- executor(Executor)：保存预测模型

In [21]:
model

name: "fc_0.tmp_4"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: -1
      dims: 10
    }
    lod_level: 0
  }
}
persistable: false

In [22]:
if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)

fluid.io.save_inference_model(dirname = model_save_dir,
                             feeded_var_names = [words.name],
                             target_vars = [model],
                             executor = exe)

print("训练模型保存完成！")

训练模型保存完成！


## Step 5: 模型预测

### 5.1 创建预测执行器

In [23]:
predict_place = fluid.CPUPlace()
predict_exe = fluid.Executor(predict_place)
predict_exe.run(fluid.default_startup_program())

[]

### 5.2 读取预测模型

load_inference_model() 函数的返回一个包含三个元素的元组
- program 预测用的程序
- feeded_var_names 一个str列表，包含需要在预测程序中提供数据的变量的名称
- target_var

In [24]:
# 从模型中获取预测程序、输入数据名称列表、分类器
[predict_program, predict_feeded_var_names, predict_target_var] = fluid.io.load_inference_model(
                                                                                dirname = model_save_dir,
                                                                                executor = predict_exe)

In [25]:
predict_target_var

[name: "save_infer_model/scale_0"
 type {
   type: LOD_TENSOR
   lod_tensor {
     tensor {
       data_type: FP32
       dims: -1
       dims: 10
     }
     lod_level: 0
   }
 }
 persistable: false]

### 5.3 获取数据

In [26]:
def get_data(sentence):
    # 读取数据字典文件
    with open(dict_path, 'r', encoding='utf-8') as f_data:
        dict_txt = eval(f_data.readlines()[0])
    
    dict_txt = dict(dict_txt)
    
    # 把字符串数据转换成列表数据
    keys = dict_txt.keys()
    
    data = []
    
    for s in sentence:
        if not s in keys:
            s = '<unk>'
        
        data.append(int(dict_txt[s]))
    
    return np.array(data, dtype = np.int64)

In [27]:
news = ["敦煌与人民大学共建敦煌文化学院两件流失海外的千年国宝将“回家”",
        "徐峥发文悼念高以翔，并严厉斥责《追我吧》节目组",
        "丁俊晖点出比赛转折点，期待英竞标赛赢下更多比赛",
        "易会满署名文章信息量大，点名监管干部",
        "11月26日甚至一手住宅成交129套涨幅约50%",
        "原北汽新能源总经理郑刚正式加盟华为",
        "学校作业多和课外班多，8成家长担忧孩子睡眠不足",
        "科学家宣称发现宇宙中第五种基本了的证据",
        "青瓦台前绝食8天后，韩总统热门人选突然不省人事",
        "沪指低开0.03%，科技股活跃权重低迷"]

In [28]:
data = []

In [29]:
for i in range(len(news)):
    data.append(get_data(news[i]))
    
# print(data)

In [30]:
# 获取每句话的单词数量
base_shape = [[len(w) for w in data]]

print(base_shape)

[[31, 23, 23, 18, 24, 17, 23, 19, 23, 19]]


In [31]:
# 生成预测数据
tensor_words = fluid.create_lod_tensor(data, base_shape, predict_place)

### 5.4 进行预测

In [32]:
result = predict_exe.run(program = predict_program,
                feed = {predict_feeded_var_names[0]: tensor_words},
                fetch_list = predict_target_var)

In [33]:
# 分类名称
names = [ '文化', '娱乐', '体育', '财经', '房产', '汽车', '教育', '科技', '国际', '证券']

In [34]:
# 获取概率最大的label
for i in range(len(data)):
    lab = np.argsort(result)[0][i][-1]
    print(news[i])
    print('预测结果标签为： {}， 名称为：{}， 概率为：{:.6f}'.format(lab, names[lab], result[0][i][lab]))
    print("*******************")

敦煌与人民大学共建敦煌文化学院两件流失海外的千年国宝将“回家”
预测结果标签为： 0， 名称为：文化， 概率为：0.940224
*******************
徐峥发文悼念高以翔，并严厉斥责《追我吧》节目组
预测结果标签为： 1， 名称为：娱乐， 概率为：0.932801
*******************
丁俊晖点出比赛转折点，期待英竞标赛赢下更多比赛
预测结果标签为： 2， 名称为：体育， 概率为：0.999573
*******************
易会满署名文章信息量大，点名监管干部
预测结果标签为： 3， 名称为：财经， 概率为：0.507758
*******************
11月26日甚至一手住宅成交129套涨幅约50%
预测结果标签为： 4， 名称为：房产， 概率为：0.561365
*******************
原北汽新能源总经理郑刚正式加盟华为
预测结果标签为： 7， 名称为：科技， 概率为：0.553036
*******************
学校作业多和课外班多，8成家长担忧孩子睡眠不足
预测结果标签为： 6， 名称为：教育， 概率为：0.998530
*******************
科学家宣称发现宇宙中第五种基本了的证据
预测结果标签为： 7， 名称为：科技， 概率为：0.879325
*******************
青瓦台前绝食8天后，韩总统热门人选突然不省人事
预测结果标签为： 8， 名称为：国际， 概率为：0.581737
*******************
沪指低开0.03%，科技股活跃权重低迷
预测结果标签为： 9， 名称为：证券， 概率为：0.683330
*******************
